# 📘 AAPL Stock Forecasting using Technical Indicators & ML

This notebook builds a complete stock price forecasting pipeline for Apple Inc. (AAPL) using:
- Technical indicators (MACD, RSI, Bollinger Bands)
- Machine learning models: Linear Regression and Random Forest
- Data visualization and evaluation metrics




In [ ]:
# ✅ Install required packages (for Google Colab only)
!pip install yfinance scikit-learn matplotlib

In [ ]:
# ✅ Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

## 1. 📥 Load Historical Data
We will use Yahoo Finance to fetch the stock price of AAPL from January 2023 to December 2024.

In [ ]:
# Download AAPL stock data
aapl = yf.download("AAPL", start="2023-01-01", end="2024-12-31")
aapl.reset_index(inplace=True)
aapl = aapl[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
aapl.dropna(inplace=True)
aapl = aapl.sort_values("Date").reset_index(drop=True)
aapl.head()

## 2. 📊 Add Technical Indicators
We will engineer several common indicators used in quantitative trading:
- MACD (Moving Average Convergence Divergence)
- RSI (Relative Strength Index)
- Bollinger Bands

In [ ]:
# MACD and Signal Line
exp1 = aapl['Close'].ewm(span=12, adjust=False).mean()
exp2 = aapl['Close'].ewm(span=26, adjust=False).mean()
aapl['MACD'] = exp1 - exp2
aapl['Signal_Line'] = aapl['MACD'].ewm(span=9, adjust=False).mean()

# RSI (Relative Strength Index)
delta = aapl['Close'].diff()
gain = delta.where(delta > 0, 0).rolling(window=14).mean()
loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
rs = gain / loss
aapl['RSI'] = 100 - (100 / (1 + rs))

# Bollinger Bands
aapl['Middle_Band'] = aapl['Close'].rolling(window=20).mean()
rolling_std = aapl['Close'].rolling(window=20).std()
aapl['Upper_Band'] = aapl['Middle_Band'] + 2 * rolling_std
aapl['Lower_Band'] = aapl['Middle_Band'] - 2 * rolling_std

# Drop missing values due to rolling window
aapl.dropna(inplace=True)
aapl.tail()

## 3. 📈 Visualize Indicators
Charts help us understand market behavior and indicator responses visually.

In [ ]:
# MACD Plot
plt.figure(figsize=(12, 4))
plt.plot(aapl['Date'], aapl['MACD'], label='MACD')
plt.plot(aapl['Date'], aapl['Signal_Line'], label='Signal Line')
plt.title('MACD and Signal Line')
plt.grid(True)
plt.legend()
plt.show()

# RSI Plot
plt.figure(figsize=(12, 4))
plt.plot(aapl['Date'], aapl['RSI'], label='RSI', color='orange')
plt.axhline(70, color='red', linestyle='--')
plt.axhline(30, color='green', linestyle='--')
plt.title('RSI Indicator')
plt.grid(True)
plt.legend()
plt.show()

# Bollinger Bands
plt.figure(figsize=(12, 4))
plt.plot(aapl['Date'], aapl['Close'], label='Close Price')
plt.plot(aapl['Date'], aapl['Upper_Band'], label='Upper Band', linestyle='--')
plt.plot(aapl['Date'], aapl['Lower_Band'], label='Lower Band', linestyle='--')
plt.title('Bollinger Bands')
plt.grid(True)
plt.legend()
plt.show()

## 4. 🤖 Build Machine Learning Models
We will predict the **Close** price using technical indicators and basic features.

In [ ]:
# Define features and target
features = ['Open', 'High', 'Low', 'Volume', 'MACD', 'Signal_Line', 'RSI',
            'Middle_Band', 'Upper_Band', 'Lower_Band']
target = 'Close'

# Split the data
X = aapl[features]
y = aapl[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Initialize models
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit models
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)

# Predict
lr_pred = lr.predict(X_test)
rf_pred = rf.predict(X_test)

## 5. 📉 Evaluate Model Performance
We compare the actual and predicted values visually and calculate RMSE.

In [ ]:
# RMSE scores
print("Linear Regression RMSE:", mean_squared_error(y_test, lr_pred, squared=False))
print("Random Forest RMSE:", mean_squared_error(y_test, rf_pred, squared=False))

# Plot Predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test.values[:100], label='Actual Close')
plt.plot(lr_pred[:100], label='Linear Regression')
plt.plot(rf_pred[:100], label='Random Forest')
plt.title('Actual vs Predicted Close Price (100 samples)')
plt.xlabel('Time Index')
plt.ylabel('Close Price')
plt.grid(True)
plt.legend()
plt.show()

---
**Project Notes:**
- Data Source: Yahoo Finance (via yFinance)
- Models: Linear Regression, Random Forest
- Indicators: MACD, RSI, Bollinger Bands
- Created for educational/demo purposes

